In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from pathlib import Path

import matplotlib.pyplot as plt
import motorlab as ml
import numpy as np
import seaborn as sns
import torch
import yaml

In [3]:
os.chdir(Path().resolve().parent)

In [6]:
runs = [
    "20250807145637",
    "20250807152324",
]
checkpoints = dict()

for run in runs:
    checkpoints[run] = torch.load(
        f"artifacts/checkpoint/poses_to_position/{run}.pt"
    )

In [ ]:
def recursive_diff(dict1, dict2, path=""):
    diffs = []
    for key in dict1:
        if key not in dict2:
            diffs.append(f"{path}{key} missing in second dict")
        else:
            val1, val2 = dict1[key], dict2[key]
            if isinstance(val1, dict) and isinstance(val2, dict):
                diffs.extend(recursive_diff(val1, val2, path + f"{key}."))
            elif isinstance(val1, torch.Tensor) and isinstance(
                val2, torch.Tensor
            ):
                if not torch.equal(val1, val2):
                    diffs.append(f"{path}{key} differs (tensor values)")
            elif val1 != val2:
                diffs.append(f"{path}{key} differs: {val1} != {val2}")
    for key in dict2:
        if key not in dict1:
            diffs.append(f"{path}{key} missing in first dict")
    return diffs


first_run = runs[0]
for run in runs[1:]:
    print(f"Differences between {first_run} and {run}:")
    diffs = recursive_diff(checkpoints[first_run], checkpoints[run])
    if diffs:
        for diff in diffs:
            print("  -", diff)
    else:
        print("  No differences found.")

Differences between 20250807145637 and 20250807152324:
  No differences found.
